In [1]:
import pandas as pd

## Data Preparation

In [44]:
#load data
df_seq=pd.read_csv('pdb_data_seq.csv')
df_properties=pd.read_csv('pdb_data_no_dups.csv')
df_total=df_seq.merge(df_properties,left_on='structureId',right_on = 'structureId')


In [3]:
df_total.columns

Index(['structureId', 'chainId', 'sequence', 'residueCount_x',
       'macromoleculeType_x', 'classification', 'experimentalTechnique',
       'macromoleculeType_y', 'residueCount_y', 'resolution',
       'structureMolecularWeight', 'crystallizationMethod',
       'crystallizationTempK', 'densityMatthews', 'densityPercentSol',
       'pdbxDetails', 'phValue', 'publicationYear'],
      dtype='object')

#### Select only protein, and filtered by top N

In [45]:
#select the data in top n by count, top 10?
df_total=df_total[df_total['macromoleculeType_x'].isin(set(['Protein']))]

count = df_total['classification'].value_counts(dropna=False)[:10]
df_selected=df_total[df_total['classification'].isin(set(count.index))]
#we want only protein
#count = df_selected['classification'].value_counts(dropna=True)[:10]

In [46]:
count

HYDROLASE                        46336
TRANSFERASE                      36424
OXIDOREDUCTASE                   34322
IMMUNE SYSTEM                    15615
LYASE                            11682
HYDROLASE/HYDROLASE INHIBITOR    11218
TRANSCRIPTION                     8919
VIRAL PROTEIN                     8495
TRANSPORT PROTEIN                 8371
VIRUS                             6972
Name: classification, dtype: int64

#### Select proteins with only one chain

In [9]:
#select proteins with only one chain in the data set
#how to justify this operation?
#df_onechain = df_selected[df_selected.groupby('structureId').structureId.transform(len) == 1]

In [48]:
test_df = df_selected[['structureId','classification','sequence']]
#test_df = df_onechain[['structureId','classification','sequence']]

Things to be done

Further select data and simplify problem, select proteins with only one chain?


Figure out how to convert sequence data into array and training model afterwards. (22 - n -n)


More models and discussion (*LSTM)

Models build on features other than sequence.



## Data balancing

In [49]:
from sklearn.utils import resample
#upsample the minorities and then take 50000 sample from all


In [56]:
df_majority = test_df[test_df['classification']=='HYDROLASE']
df_minority = test_df[test_df['classification']=='HYDROLASE']
 
# Upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=46336,    # to match majority class
                                 random_state=123) # reproducible results

In [57]:
keylist = list(count.index)
datalist = [0]*9
for i in range(1,10):
    df_minority = test_df[test_df['classification']==keylist[i]]
    datalist[i-1] = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=46336,    # to match majority class
                                 random_state=123) # reproducible results
    #print(len(df_majority))
    df_majority = pd.concat([df_majority, datalist[i-1]])
data = df_majority.sample(50000)
    

46336
92672
139008
185344
231680
278016
324352
370688
417024


## Model Training

In [60]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier,ExtraTreesClassifier

In [61]:
#optional, take part of the data for faster verification
#data = test_df#.sample(50000)

#need to remove nulls
data = data.dropna()
X_train, X_test,y_train,y_test = \
train_test_split(data['sequence'], data['classification'], test_size = 0.2, random_state = 1)

In [11]:
count

RIBOSOME                         60710
HYDROLASE                        47833
TRANSFERASE                      37726
OXIDOREDUCTASE                   35114
IMMUNE SYSTEM                    15989
LYASE                            11871
HYDROLASE/HYDROLASE INHIBITOR    11262
VIRUS                            10832
TRANSCRIPTION                    10564
VIRAL PROTEIN                     8875
Name: classification, dtype: int64

#### Feature Extraction From Sequence Data

In [64]:
#vectorize data, prepare for building models
#Convert a collection of text documents to a matrix of token counts
#seems has nothing to do with sequence but only with the frequency

#ngram is a parameter we need to focus on, 

vect = CountVectorizer(analyzer = 'char_wb', ngram_range = (1,1))
#vect = CountVectorizer(analyzer = 'char_wb')

# Fit and Transform CountVectorizer
#occasionally may meet np.nan error
vect.fit(X_train)
X_train_df = vect.transform(X_train)
X_test_df = vect.transform(X_test)

#to store the results for different mothods
prediction = dict()

#### Naive Bayes

In [65]:
model = MultinomialNB()
model.fit(X_train_df, y_train)
#test on test set
NB_pred = model.predict(X_test_df)
prediction["MultinomialNB"] = accuracy_score(NB_pred, y_test)
print( prediction['MultinomialNB'])

0.3754


#### adaboost

In [66]:
model = AdaBoostClassifier()
model.fit(X_train_df,y_train)
ADA_pred = model.predict(X_test_df)
prediction["Adaboost"] = accuracy_score(ADA_pred , y_test)
print(prediction["Adaboost"])

0.4017


#### RandomForestClassifier

In [67]:
model = RandomForestClassifier()
model.fit(X_train_df,y_train)
ADA_pred = model.predict(X_test_df)
prediction["Random_Forest"] = accuracy_score(ADA_pred , y_test)
print(prediction["Random_Forest"])

C:\Users\Weinan\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.8701


Iterate through models

In [42]:
models = [MultinomialNB(),AdaBoostClassifier(),RandomForestClassifier()]

In [43]:
model_name = ['NB','adaboost','randomForest']

In [44]:
models = dict(zip(model_name,models))

In [ ]:
from tqdm import tqdm
predictions=[]
for key,model in models:
    
    for ngram in tqdm(range(1,6)):
        vect = CountVectorizer(analyzer = 'char_wb', ngram_range = (ngram,ngram))
        vect.fit(X_train)
        X_train_df = vect.transform(X_train)
        X_test_df = vect.transform(X_test)
        
        model.fit(X_train_df,y_train)
        ADA_pred = model.predict(X_test_df)
        if key not in prediction:
            prediction[key] = [accuracy_score(ADA_pred , y_test)]
        else:
            prediction[key].append(accuracy_score(ADA_pred , y_test))  